<a href="https://colab.research.google.com/github/joshuaword2alt/joshuaword2alt.github.io/blob/main/neuralfunkv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WaveGAN interactive notebook

This notebook allows you to play with WaveGAN on (free) cloud GPUs.

To get started, go to: `Runtime > Change runtime type > Hardware accelerator > GPU ... Save`

Then, simply run the cells below (`Ctrl + Enter` while highlighting a cell)

# **NeuralFunk vol.2**

Project by Ivan.Garkavy@skoltech.ru (with help from Marsel.Ishimbaev@skoltech.ru)

Demo video: https://youtu.be/oHKLAkCNrj8

I trained WaveGAN neural network (https://github.com/chrisdonahue/wavegan) on 7500 vintage drumloops (breaks) from around 2000 old recordings.

All the loops were fit to 120bpm tempo and sampled at 32768Hz for the dataset.

The trained model is able to generate either non-existing breaks or new drumloops with elements of existing breaks, and smoothly morph between generated loops.

This result is about to be improved in the future.

**Link to the trained model can be found in this colab notebook.**



In [ ]:
!nvidia-smi

In [ ]:
%tensorflow_version 1.15
import tensorflow as tf
#Then check the version:
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [ ]:
# Confirm GPU is running
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
if len(get_available_gpus()) == 0:
  for i in range(4):
    print('WARNING: Not running on a GPU! See above for faster generation')

In [ ]:
# mount google drive
# you'll need to give permission for Google Colab (not me) to access your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

link to the model: https://drive.google.com/drive/folders/1avrLFIDJoY1UdOpHjUG1J02KuLJvrW5n?usp=sharing

add shortcut to this folder to your google drive


In [ ]:
# downloading the model from your google drive (2GB)
# it takes about 1 min
!cp -r -v "/content/gdrive/MyDrive/neuralfunkv2/model" "/content/"
# (if it fails to find the folder, and you've made a shortcut,
# try changing "MyDrive" to "My Drive" everywhere in the notebook)

'/content/gdrive/MyDrive/neuralfunkv2/model' -> '/content/model'
'/content/gdrive/MyDrive/neuralfunkv2/model/model.ckpt-18637.data-00000-of-00001' -> '/content/model/model.ckpt-18637.data-00000-of-00001'
'/content/gdrive/MyDrive/neuralfunkv2/model/model.ckpt-18637.index' -> '/content/model/model.ckpt-18637.index'
'/content/gdrive/MyDrive/neuralfunkv2/model/model.ckpt-18637.meta' -> '/content/model/model.ckpt-18637.meta'
'/content/gdrive/MyDrive/neuralfunkv2/model/args.txt' -> '/content/model/args.txt'
'/content/gdrive/MyDrive/neuralfunkv2/model/graph.pbtxt' -> '/content/model/graph.pbtxt'
'/content/gdrive/MyDrive/neuralfunkv2/model/checkpoint' -> '/content/model/checkpoint'
'/content/gdrive/MyDrive/neuralfunkv2/model/infer' -> '/content/model/infer'
'/content/gdrive/MyDrive/neuralfunkv2/model/infer/infer.pbtxt' -> '/content/model/infer/infer.pbtxt'
'/content/gdrive/MyDrive/neuralfunkv2/model/infer/infer.meta' -> '/content/model/infer/infer.meta'


In [ ]:
# Load the model
tf.reset_default_graph()
saver = tf.train.import_meta_graph('/content/model/infer/infer.meta')
graph = tf.get_default_graph()
sess = tf.InteractiveSession()
saver.restore(sess, f'/content/model/model.ckpt-18637')
dim = 100
break_len = 65536

z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')

INFO:tensorflow:Restoring parameters from /content/model/model.ckpt-18637


In [ ]:
import numpy as np
from IPython.display import display, Audio
from google.colab import files
import scipy.io.wavfile
import matplotlib.pyplot as plt
%matplotlib inline
!mkdir "/content/neuralfunk examples"

def generate_trajectory(n_iter, _z0=None, mov_last=None, jump=0.3, smooth=0.3, include_z0=True):
    _z = np.empty((n_iter + int(not include_z0), dim))
    _z[0] = _z0 if _z0 is not None else np.random.random(dim)*2-1
    mov = mov_last if mov_last is not None else (np.random.random(dim)*2-1)*jump
    for i in range(1, len(_z)):
        mov = mov*smooth + (np.random.random(dim)*2-1)*jump*(1-smooth)
        mov -= (np.abs(_z[i-1] + mov) > 1) * 2 * mov
        _z[i] = _z[i-1] + mov
    return _z[-n_iter:], mov

# Configuring is done, next cells are generation of results

*In this notebook, files are previewed at 39936 hz rate (146.25 bpm tempo, but saved at 44100 hz (=161.4990234375 bpm tempo).*

*You may change these numbers in code (for example, 32768 hz for 120 bpm), but you will probably need some standard sample rate (44100 etc) for files to be able to open them your DAW.*

*In this case you may specify either this clumsy bpm value or the number of bars in DAW to fit the audio to any tempo you like.*

In [ ]:
# Generate random breaks and display audio

# CHANGE THIS to change number of examples generated
n_generate = 30

# Sample latent vectors
seed = 666 # change this seed to generate different set of breaks
np.random.seed(seed)
_z = (np.random.rand(n_generate, dim) * 2.) - 1.
_G_z = sess.run(G_z, {z: _z})[:,:,0]

# display(Audio(_G_z.flatten(), rate=39936)) # display all in one audio

for i in range(n_generate): # display separate audio for each break
  print(i)
  display(Audio(np.tile(_G_z[i], 2), rate=39936)) # change rate for different tempo

In [ ]:
# save to VM local memory (you'll need to copy your files to google drive in the last cell)
filename = f"/content/neuralfunk examples/neuralfunkv2_gen_s{seed}_n{n_generate}.wav"
scipy.io.wavfile.write(filename, 44100, _G_z.flatten()) # change rate for different tempo

In [ ]:
# download the file directly from browser
files.download(filename)

In [ ]:
# Interpolate between two breaks

# CHANGE THIS to change number of steps
n_generate = 7

seed = 888 # change this seed to generate different start and end breaks
np.random.seed(seed)
_z12 = (np.random.rand(2, dim) * 2.) - 1.
grad = np.linspace(0, 1, n_generate).reshape(-1, 1)
_z = _z12[0] * (1-grad) + _z12[1] * grad
_G_z = sess.run(G_z, {z: _z})[:,:,0]
display(Audio(_G_z.flatten(), rate=39936)) # display all in one audio

In [ ]:
# save to VM local memory (you'll need to copy your files to google drive in the last cell)
filename = f"/content/neuralfunk examples/neuralfunkv2_morph_s{seed}_n{n_generate}.wav"
scipy.io.wavfile.write(filename, 44100, _G_z.flatten())

In [ ]:
# download the file directly from browser
files.download(filename)

In [ ]:
# Move along local random trajectory in the latent space
# (endless generative "improvisation" with smoothly changing breaks)
n_generate = 128

jump = 0.4 # factor of distance between adjacent trajectory points
           # (speed of changing)
smooth = 0.25 # smoothing the trajectory turns, [0, 1]

seed1 = 50 # change this to change starting point
np.random.seed(seed1)
_z0 = np.random.random(dim) * 2 - 1

seed2 = 3 # change this to change trajectory
np.random.seed(seed2)
_z, mov_last = generate_trajectory(n_generate, _z0=_z0, include_z0=True,
                                    jump=jump, smooth=smooth)
_G_z = sess.run(G_z, {z: _z})[:,:,0]

# show the trajectory in first 2 dimensions
plt.scatter(_z[:, 0], _z[:, 1], s=10, color='blue')
plt.plot(_z[:, 0], _z[:, 1], color='green')
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.show()

display(Audio(_G_z.flatten(), rate=39936))

In [ ]:
# save to VM local memory (you'll need to copy your files to google drive in the last cell)
# this demo is 2 GB!
filename = f"/content/neuralfunk examples/neuralfunkv2_trajectory_sa{seed1}_sb{seed2}_j{jump}_sm{smooth}_n{n_generate}.wav"
scipy.io.wavfile.write(filename, 44100, _G_z.flatten())

In [ ]:
# download the file directly from browser.
files.download(filename)

In [ ]:
# Move along LONG local random trajectory in the latent space
# (endless generative "improvisation" with smoothly changing breaks)
# (recreates the demo from youtube)

# generate trajectory of part_len * n_parts breaks
n_parts = 16
part_len = 275

jump = 0.4
smooth = 0.25

seed_stereo = 9
np.random.seed(seed_stereo)
stereo_width = 2e-2
stereo_diff = (np.random.random((1, dim)) * 2 - 1)*stereo_width

seed1 = 50
np.random.seed(seed1)
_z0 = np.random.random(dim) * 2 - 1
_G_z_full = np.zeros((n_parts, part_len, break_len, 2), dtype='float32')

seed2 = 3
np.random.seed(seed2)
_z_full = np.empty((n_parts, part_len, dim), dtype=_z0.dtype)
for part in range(n_parts):
    if part == 0:
        _z, mov_last = generate_trajectory(part_len, _z0=_z0, include_z0=True,
                                           jump=jump, smooth=smooth)
    else:
        _z, mov_last = generate_trajectory(part_len, _z0=_z[-1], include_z0=False, mov_last=mov_last,
                                           jump=jump, smooth=smooth)
    _z_full[part] = _z
    _G_z = sess.run(G_z, {z: _z})[:,:,0]
    _G_z_full[part, :, :, 0] = _G_z
    # slightly changing latent vector to change the break slightly
    # and use this as stereo difference
    _z_2 = _z + stereo_diff
    _G_z_2 = sess.run(G_z, {z: _z_2})[:,:,0]
    _G_z_full[part, :, :, 1] = _G_z_2
    print(f"{part+1} / {n_parts} parts ready")
_G_z_full_flatten = _G_z_full.reshape(-1, 2)

In [ ]:
# display first few bars of result
n_display = 64 # runtime disconnects when trying to preview too large audio track
display(Audio((_G_z_full_flatten[:break_len*n_display, 0], _G_z_full_flatten[:break_len*n_display, 1]), rate=39936))

In [ ]:
# save the whole demo to local VM memory
filename = f"/content/neuralfunk examples/neuralfunkv2_trajectory_sa{seed1}_sb{seed2}_j{jump}_sm{smooth}_n{n_parts*part_len}.wav"
scipy.io.wavfile.write(filename, 44100, _G_z_full_flatten)

In [ ]:
# save all audio files to your google drive (to the "neuralfunk examples" folder in its root)
!cp -r -v "/content/neuralfunk examples" "/content/gdrive/MyDrive/" && rm /content/neuralfunk\ examples/*

'/content/neuralfunk examples' -> '/content/gdrive/MyDrive/neuralfunk examples'
'/content/neuralfunk examples/neuralfunkv2_gen_s666_n30.wav' -> '/content/gdrive/MyDrive/neuralfunk examples/neuralfunkv2_gen_s666_n30.wav'
'/content/neuralfunk examples/neuralfunkv2_morph_s888_n7.wav' -> '/content/gdrive/MyDrive/neuralfunk examples/neuralfunkv2_morph_s888_n7.wav'
'/content/neuralfunk examples/neuralfunkv2_trajectory_sa50_sb3_j0.4_sm0.25_n128.wav' -> '/content/gdrive/MyDrive/neuralfunk examples/neuralfunkv2_trajectory_sa50_sb3_j0.4_sm0.25_n128.wav'
'/content/neuralfunk examples/neuralfunkv2_trajectory_sa50_sb3_j0.4_sm0.25_n_4400.wav' -> '/content/gdrive/MyDrive/neuralfunk examples/neuralfunkv2_trajectory_sa50_sb3_j0.4_sm0.25_n_4400.wav'
